In [1]:
pip install tensorflow numpy pillow


In [6]:
import os
import json
import tarfile
import numpy as np
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 데이터셋 경로 설정
data_dir = 'C:/Users/tata/Desktop/딸기 데이터/설향/01.데이터'

# JSON 파일에서 생장단계 읽어오기
with open(os.path.join(data_dir, 'data-text.json'), 'r', encoding='utf-8') as f:
    labels_dict = json.load(f)

# 이미지 경로와 라벨 수집
image_paths = []
labels = []

for folder_name in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder_name)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                image_paths.append(os.path.join(folder_path, filename))
                labels.append(labels_dict[folder_name])

# 데이터셋 분할
X_train, X_val, y_train, y_val = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# 이미지 데이터 불러오기 및 전처리
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))  # ResNet50 입력 크기에 맞게 리사이징
    img_array = np.array(img)  # 이미지를 넘파이 배열로 변환
    img_array = img_array / 255.0  # 이미지 스케일링 (0 ~ 1 사이 값으로 변환)
    return img_array

# 데이터셋 생성
X_train_processed = np.array([load_and_preprocess_image(img_path) for img_path in X_train])
X_val_processed = np.array([load_and_preprocess_image(img_path) for img_path in X_val])
y_train_encoded = np.array(y_train)
y_val_encoded = np.array(y_val)

# ResNet50 모델 불러오기 (pre-trained weights 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 구성
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
output = Dense(len(set(labels)), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# 모델 컴파일 및 학습
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_processed, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_val_processed, y_val_encoded))


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/tata/Desktop/딸기 데이터/설향/01.데이터\\data-text.json'